In [2]:
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import keras_metrics
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.convolutional import SeparableConv2D
from keras.layers.normalization import BatchNormalization



import math



In [3]:
df_metadata = pd.read_csv(filepath_or_buffer='..\Images\HAM10000_metadata.csv')


In [4]:
df_metadata.groupby('dx').count()

,lesion_id,image_id,dx_type,age,sex,localization
dx,,,,,,
akiec,327,327,327,327,327,327
bcc,514,514,514,514,514,514
bkl,1099,1099,1099,1089,1099,1099
df,115,115,115,115,115,115
mel,1113,1113,1113,1111,1113,1113
nv,6705,6705,6705,6660,6705,6705
vasc,142,142,142,142,142,142


In [5]:
#checking to see what the unique categories are
df_metadata['dx'].unique()

array(['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec'], dtype=object)

In [6]:
#keeping only relevant data, image filepath, and category
df=df_metadata.loc[:,['image_id','dx']]

#add .jpg extension to filepaths for every sample in image_id column
df['image_id']=df['image_id'].apply(lambda x: x+'.jpg' )
df.head()

,image_id,dx
0,ISIC_0027419.jpg,bkl
1,ISIC_0025030.jpg,bkl
2,ISIC_0026769.jpg,bkl
3,ISIC_0025661.jpg,bkl
4,ISIC_0031633.jpg,bkl


In [7]:
df['dx'].groupby(df['dx']).count()

dx
akiec     327
bcc       514
bkl      1099
df        115
mel      1113
nv       6705
vasc      142
Name: dx, dtype: int64

In [8]:
#call bkl, mel and bcc cancer (1) and the rest not cancer (0)


# WHY DOES THIS CODE WORK BUT THE FOLLOWING DOES!?!?!??!
def binary_coder(sample):
    if (sample =='mel') or (sample=='bcc') or (sample=="akiec"):
        sample='1_cancer'
    else:
        sample='0_not_cancer'
        
    return sample
    

# def binary_coder(sample):
#     if (sample =='mel') or (sample=='bcc') or (sample=="akiec"):
#         sample='cancer'
#         return sample

#     else:
#         sample='not cancer'
#         return sample
    
df['dx'] = df['dx'].apply(binary_coder) 


# df.head()

In [9]:
df['dx'].groupby(df['dx']).count()

dx
0_not_cancer    8061
1_cancer        1954
Name: dx, dtype: int64

# Undersampling

In [10]:
instances_of_cancer = len(df.loc[df['dx']=='cancer'])
instances_of_not_cancer = len(df.loc[df['dx']=='not_cancer'])

#ratio of cancer to not cancer

# instances_of_not 

In [11]:
df['dx'].groupby(df['dx']).count()

dx
0_not_cancer    8061
1_cancer        1954
Name: dx, dtype: int64

In [12]:
balanced_df=df.drop(df[df['dx']=='0_not_cancer'].sample(frac=.7576).index)

In [13]:
balanced_df['dx'].groupby(balanced_df['dx']).count()

dx
0_not_cancer    1954
1_cancer        1954
Name: dx, dtype: int64

In [14]:
df_train, df_test =train_test_split(balanced_df, train_size=.8, random_state =2, stratify=balanced_df['dx'])



C:\Users\petra\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
df_train.head()
df_train.shape

(3126, 2)

In [16]:
df_test.shape

(782, 2)

In [17]:
batch_size = 10
image_dimensions = (25,25)

train_gen = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen=train_gen.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='categorical', batch_size= batch_size)
test_gen = test_gen.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='categorical', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [18]:
train_gen.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [19]:
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:


# def recall(y_true, y_pred):
#     return recall_score(y_true, y_pred, average = 'micro')
input_into_first_layer = (25,25,1)

model = Sequential()

model.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2) ))

model.add(Conv2D(64, (3, 3), activation ='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model.add(Dense(2, activation ='softmax'))
# model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall, f1])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 23, 23, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_

In [18]:
model.fit_generator(
        train_gen,
        steps_per_epoch=3126,
        epochs=2)
#         class_weight = {0:1,

Instructions for updating:
Use tf.cast instead.
Epoch 1/2


KeyboardInterrupt: 

In [ ]:
#SWITCHING from clas_mode=binary loss = sparse_cat_crossE and etrics = binary to categorical leads to better results sparse leads to betet

In [26]:
model.evaluate_generator(test_gen,steps =len(test_gen))

[0.5745448469353454, 0.700767267466811, 0.700767267466811, 0.7007672269173595]

# Iteration 2: Trying 1 node in the output with a sigmoid activation fucntion

class_mode= binary
loss= binary_crossentropy
1 sigmoid activation node in output


In [30]:
batch_size = 10
image_dimensions = (25,25)


train_gen2 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen2 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen2=train_gen2.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen2= test_gen2.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [31]:
model2 = Sequential()

model2.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Conv2D(32, (3, 3),activation = 'relu'))
model2.add(MaxPooling2D(pool_size=(2, 2) ))

model2.add(Conv2D(64, (3, 3), activation ='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model2.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model2.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model2.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 23, 23, 32)        320       
_________________________________________________________________
activation_3 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 1, 64)          0         
__________

In [32]:
model2.fit_generator(
        train_gen2,
        steps_per_epoch=3126,
        epochs=2)
#         class_weight = {0:1,

Epoch 1/2
3126/3126 [==============================] - 407s 130ms/step - loss: 0.6693 - binary_accuracy: 0.5691 - recall: 0.5844 - precision: 0.4791 - f1: 0.4759
Epoch 2/2
3126/3126 [==============================] - 420s 134ms/step - loss: 0.5734 - binary_accuracy: 0.7062 - recall: 0.7331 - precision: 0.7012 - f1: 0.6857


In [258]:
model2.evaluate_generator(test_gen2,steps =len(test_gen))


[0.5235555291175842,
 0.7433333337306977,
 0.6808730244636536,
 0.7615873058636983,
 0.6965946644544602]

## Iteration Three

class_mode=sparse
loss=binary_crossentropy
outut layer: 1 sigmoid activation

In [259]:
batch_size = 10
image_dimensions = (25,25)

train_gen3 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen3 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen3=train_gen3.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='sparse', batch_size= batch_size)
test_gen3= test_gen3.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='sparse', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [35]:
model3 = Sequential()

model3.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(32, (3, 3),activation = 'relu'))
model3.add(MaxPooling2D(pool_size=(2, 2) ))

model3.add(Conv2D(64, (3, 3), activation ='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model3.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model3.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 23, 23, 32)        320       
_________________________________________________________________
activation_9 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 1, 1, 64)          0         
__________

In [36]:
model3.fit_generator(
        train_gen3,
        steps_per_epoch=3126,
        epochs=2)
#         class_weight = {0:1,

Epoch 1/2
3126/3126 [==============================] - 566s 181ms/step - loss: 0.6862 - binary_accuracy: 0.5283 - recall: 0.6376 - precision: 0.4381 - f1: 0.4771
Epoch 2/2
3126/3126 [==============================] - 548s 175ms/step - loss: 0.5943 - binary_accuracy: 0.6814 - recall: 0.6394 - precision: 0.7029 - f1: 0.6322


In [49]:
#len(test_gen_3) * batch_size previously define is about the number of testing samples we have
model3.evaluate_generator(test_gen3, steps = len(test_gen3))


[0.5498295507924941,
 0.7365728889584846,
 0.678129950600207,
 0.7425658689130603,
 0.6799075242579745]

In [52]:
train_gen3.class_indices

{'cancer': 0, 'not cancer': 1}

# Iteration 4

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:1.5}

In [261]:
batch_size = 10
image_dimensions = (25,25)

train_gen4 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen4 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen4=train_gen4.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen4= test_gen4.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [262]:
train_gen4.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [263]:
model4 = Sequential()

model4.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))

model4.add(Conv2D(32, (3, 3),activation = 'relu'))
model4.add(MaxPooling2D(pool_size=(2, 2) ))

model4.add(Conv2D(64, (3, 3), activation ='relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))

model4.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model4.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model4.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model4.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 23, 23, 32)        320       
_________________________________________________________________
activation_15 (Activation)   (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 1, 1, 64)          0         
__________

In [265]:
model4.fit_generator(
        train_gen4,
        steps_per_epoch=3126,
        epochs=2,
        class_weight = {0:1,
                        1:1.5}
)

Epoch 1/2
3126/3126 [==============================] - 422s 135ms/step - loss: 0.8136 - binary_accuracy: 0.5486 - recall: 0.9268 - precision: 0.5349 - f1: 0.6523
Epoch 2/2
3126/3126 [==============================] - 424s 136ms/step - loss: 0.7074 - binary_accuracy: 0.6843 - recall: 0.8366 - precision: 0.6494 - f1: 0.7067


In [266]:
#len(test_gen_3) * batch_size previously define is about the number of testing samples we have
model4.evaluate_generator(test_gen4, steps = len(test_gen4))


[0.5803747610820224,
 0.6969309472062094,
 0.8654853261035421,
 0.6500172622292243,
 0.7240466039503932]

# Iteration 5

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}

In [43]:
batch_size = 10
image_dimensions = (25,25)

train_gen5 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen5 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen5=train_gen5.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen5= test_gen5.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [44]:
train_gen5.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [45]:
model5 = Sequential()

model5.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))

model5.add(Conv2D(32, (3, 3),activation = 'relu'))
model5.add(MaxPooling2D(pool_size=(2, 2) ))

model5.add(Conv2D(64, (3, 3), activation ='relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))

model5.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model5.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model5.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model5.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 23, 23, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 1, 1, 64)          0         
__________

In [46]:
model5.fit_generator(
        train_gen5,
        steps_per_epoch=3126,
        epochs=2,
        class_weight = {0:1,
                        1:2.5}
)

Epoch 1/2
3126/3126 [==============================] - 418s 134ms/step - loss: 0.9940 - binary_accuracy: 0.5388 - recall: 0.9767 - precision: 0.5264 - f1: 0.6650
Epoch 2/2
3126/3126 [==============================] - 446s 143ms/step - loss: 0.8492 - binary_accuracy: 0.6674 - recall: 0.9285 - precision: 0.6159 - f1: 0.7215


In [47]:
#len(test_gen_3) * batch_size previously define is about the number of testing samples we have
model5.evaluate_generator(test_gen5, steps = len(test_gen5))


[0.7447367730881552,
 0.6355498771914436,
 0.9773576082780843,
 0.5807859461821253,
 0.7154763806658937]

# Iteration 6


class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}
increase epoch to 30                      
                        

In [33]:
batch_size = 10
image_dimensions = (25,25)

train_gen6 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen6 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen6=train_gen6.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen6= test_gen6.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [34]:
train_gen6.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [35]:
model6 = Sequential()

model6.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model6.add(Activation('relu'))
model6.add(MaxPooling2D(pool_size=(2, 2)))

model6.add(Conv2D(32, (3, 3),activation = 'relu'))
model6.add(MaxPooling2D(pool_size=(2, 2) ))

model6.add(Conv2D(64, (3, 3), activation ='relu'))
model6.add(MaxPooling2D(pool_size=(2, 2)))

model6.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model6.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model6.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model6.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 23, 23, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 1, 1, 64)          0         
__________

In [37]:
model6.fit_generator(
        train_gen6,
        steps_per_epoch=3126,
        epochs=30,
        class_weight = {0:1,
                        1:2.5}
)




Epoch 1/30
3126/3126 [==============================] - 423s 135ms/step - loss: 0.9106 - binary_accuracy: 0.6090 - recall: 0.9437 - precision: 0.5747 - f1: 0.6932
Epoch 2/30
3126/3126 [==============================] - 430s 137ms/step - loss: 0.8326 - binary_accuracy: 0.6803 - recall: 0.9222 - precision: 0.6255 - f1: 0.7261
Epoch 3/30
3126/3126 [==============================] - 426s 136ms/step - loss: 0.8105 - binary_accuracy: 0.6874 - recall: 0.9249 - precision: 0.6326 - f1: 0.7311
Epoch 4/30
3126/3126 [==============================] - 422s 135ms/step - loss: 0.7935 - binary_accuracy: 0.6991 - recall: 0.9245 - precision: 0.6420 - f1: 0.7395
Epoch 5/30
3126/3126 [==============================] - 420s 134ms/step - loss: 0.7758 - binary_accuracy: 0.7082 - recall: 0.9282 - precision: 0.6486 - f1: 0.7458
Epoch 6/30
3126/3126 [==============================] - 414s 133ms/step - loss: 0.7700 - binary_accuracy: 0.7124 - recall: 0.9267 - precision: 0.6519 - f1: 0.7475
Epoch 7/30
3126/3126 [

In [41]:
model6.evaluate_generator(test_gen6, steps = len(test_gen6))


[0.7833812721168903,
 0.6867007717604527,
 0.9404152965606631,
 0.623609597878078,
 0.7389177395712079]

# Iteration 7

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}
epoch 10
100 x 100 images

In [63]:
batch_size = 10
image_dimensions = (100,100)
input_into_first_layer = (100,100,1)


train_gen7 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen7 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen7=train_gen7.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen7= test_gen7.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [64]:
train_gen7.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [65]:
model7 = Sequential()

model7.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model7.add(Activation('relu'))
model7.add(MaxPooling2D(pool_size=(2, 2)))

model7.add(Conv2D(32, (3, 3),activation = 'relu'))
model7.add(MaxPooling2D(pool_size=(2, 2) ))

model7.add(Conv2D(64, (3, 3), activation ='relu'))
model7.add(MaxPooling2D(pool_size=(2, 2)))

model7.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model7.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model7.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model7.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model7.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 98, 98, 32)        320       
_________________________________________________________________
activation_11 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 10, 10, 64)        0         
__________

In [66]:
model7.fit_generator(
        train_gen7,
        steps_per_epoch=3126,
        epochs=10,
        class_weight = {0:1,
                        1:2.5}
)




Epoch 1/10
3126/3126 [==============================] - 452s 145ms/step - loss: 0.9512 - binary_accuracy: 0.5595 - recall: 0.9705 - precision: 0.5387 - f1: 0.6736
Epoch 2/10
3126/3126 [==============================] - 441s 141ms/step - loss: 0.8145 - binary_accuracy: 0.6963 - recall: 0.9226 - precision: 0.6401 - f1: 0.7367
Epoch 3/10
3126/3126 [==============================] - 444s 142ms/step - loss: 0.7666 - binary_accuracy: 0.7157 - recall: 0.9316 - precision: 0.6533 - f1: 0.7505
Epoch 4/10
3126/3126 [==============================] - 498s 159ms/step - loss: 0.7436 - binary_accuracy: 0.7268 - recall: 0.9338 - precision: 0.6638 - f1: 0.7590
Epoch 5/10
3126/3126 [==============================] - 842s 269ms/step - loss: 0.7182 - binary_accuracy: 0.7394 - recall: 0.9390 - precision: 0.6725 - f1: 0.7676
Epoch 6/10
3126/3126 [==============================] - 567s 182ms/step - loss: 0.7030 - binary_accuracy: 0.7443 - recall: 0.9384 - precision: 0.6784 - f1: 0.7715
Epoch 7/10
3126/3126 [

In [67]:
model7.evaluate_generator(test_gen7, steps = len(test_gen7))


[0.688211403291701,
 0.7161125312833225,
 0.9497929608730404,
 0.6471298748193799,
 0.7571639774553002]

# Iteration 8

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}
epoch 8
150 x 150 images

In [22]:
batch_size = 10
image_dimensions = (150,150)
input_into_first_layer = (150,150,1)


train_gen8 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen8 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen8=train_gen8.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen8= test_gen8.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [23]:
train_gen8.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [24]:
model8 = Sequential()

model8.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
model8.add(Activation('relu'))
model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Conv2D(32, (3, 3),activation = 'relu'))
model8.add(MaxPooling2D(pool_size=(2, 2) ))

model8.add(Conv2D(64, (3, 3), activation ='relu'))
model8.add(MaxPooling2D(pool_size=(2, 2)))

model8.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model8.add(Dense(64, activation ='relu'))
# model.add(Dropout(0.5))
model8.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))

model8.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model8.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
activation_3 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 17, 17, 64)        0         
__________

In [25]:
model8.fit_generator(
        train_gen8,
        steps_per_epoch=3126,
        epochs=8,
        class_weight = {0:1,
                        1:2.5}
)




Epoch 1/8
3126/3126 [==============================] - 479s 153ms/step - loss: 1.0081 - binary_accuracy: 0.5047 - recall: 0.9971 - precision: 0.5028 - f1: 0.6531
Epoch 2/8
3126/3126 [==============================] - 471s 151ms/step - loss: 0.8720 - binary_accuracy: 0.6472 - recall: 0.9363 - precision: 0.6004 - f1: 0.7114
Epoch 3/8
3126/3126 [==============================] - 471s 151ms/step - loss: 0.7657 - binary_accuracy: 0.7182 - recall: 0.9334 - precision: 0.6555 - f1: 0.7526
Epoch 4/8
3126/3126 [==============================] - 472s 151ms/step - loss: 0.7203 - binary_accuracy: 0.7361 - recall: 0.9384 - precision: 0.6703 - f1: 0.7657
Epoch 5/8
3126/3126 [==============================] - 476s 152ms/step - loss: 0.6789 - binary_accuracy: 0.7532 - recall: 0.9406 - precision: 0.6865 - f1: 0.7780
Epoch 6/8
3126/3126 [==============================] - 475s 152ms/step - loss: 0.6464 - binary_accuracy: 0.7692 - recall: 0.9432 - precision: 0.7007 - f1: 0.7885
Epoch 7/8
3126/3126 [=======

In [27]:
model8.evaluate_generator(test_gen8, steps = len(test_gen8))


[0.7003274725754852,
 0.7212276231602329,
 0.9380201357404899,
 0.661607406099739,
 0.7611076059701193]

# Iteration 9


class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}
epoch 10
150 x 1500 images

Architecture from: https://www.pyimagesearch.com/2019/02/18/breast-cancer-classification-with-keras-and-deep-learning/

In [23]:
batch_size = 10
image_dimensions = (150,150)
input_into_first_layer = (150,150,1)


train_gen9 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen9 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen9=train_gen9.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen9= test_gen9.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [24]:
train_gen9.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [27]:

# model9.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
# model9.add(Activation('relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Conv2D(32, (3, 3),activation = 'relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2) ))

# model9.add(Conv2D(64, (3, 3), activation ='relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model9.add(Dense(64, activation ='relu'))
# # model.add(Dropout(0.5))
# model9.add(Dense(1, activation ='sigmoid'))

# model9.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['binary_accuracy',recall,precision, f1])

# model9.summary()

# CONV => RELU => POOL
chanDim = -1

model9 = Sequential()

model9.add(SeparableConv2D(32, (3, 3), padding="same",
input_shape=input_into_first_layer))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(MaxPooling2D(pool_size=(2, 2)))
model9.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 2
model9.add(SeparableConv2D(64, (3, 3), padding="same"))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(SeparableConv2D(64, (3, 3), padding="same"))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(MaxPooling2D(pool_size=(2, 2)))
model9.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 3
model9.add(SeparableConv2D(128, (3, 3), padding="same"))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(SeparableConv2D(128, (3, 3), padding="same"))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(SeparableConv2D(128, (3, 3), padding="same"))
model9.add(Activation("relu"))
model9.add(BatchNormalization(axis=chanDim))
model9.add(MaxPooling2D(pool_size=(2, 2)))
model9.add(Dropout(0.25))

model9.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model9.add(Dense(256, activation ='relu'))
model9.add(BatchNormalization())

model9.add(Dropout(0.5))
model9.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))


model9.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model9.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_3 (Separabl (None, 150, 150, 32)      73        
_________________________________________________________________
activation_4 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 75, 75, 64)        2400      
___________________________

In [28]:
model9.fit_generator(
        train_gen9,
        steps_per_epoch=3126,
        epochs=8,
        class_weight = {0:1,
                        1:2.5}
)

Epoch 1/8
3126/3126 [==============================] - 648s 207ms/step - loss: 1.0004 - binary_accuracy: 0.6023 - recall: 0.8813 - precision: 0.5710 - f1: 0.6731
Epoch 2/8
3126/3126 [==============================] - 521s 167ms/step - loss: 0.9057 - binary_accuracy: 0.6359 - recall: 0.9145 - precision: 0.5930 - f1: 0.6998
Epoch 3/8
3126/3126 [==============================] - 490s 157ms/step - loss: 0.8760 - binary_accuracy: 0.6525 - recall: 0.9229 - precision: 0.6058 - f1: 0.7109
Epoch 4/8
3126/3126 [==============================] - 502s 160ms/step - loss: 0.8590 - binary_accuracy: 0.6669 - recall: 0.9215 - precision: 0.6161 - f1: 0.7190
Epoch 5/8
3126/3126 [==============================] - 481s 154ms/step - loss: 0.8712 - binary_accuracy: 0.6587 - recall: 0.9199 - precision: 0.6101 - f1: 0.7132
Epoch 6/8
3126/3126 [==============================] - 477s 153ms/step - loss: 0.8512 - binary_accuracy: 0.6701 - recall: 0.9253 - precision: 0.6184 - f1: 0.7221
Epoch 7/8
3126/3126 [=======

In [50]:
model9.evaluate_generator(test_gen9, steps = len(test_gen9))


[0.5799204055839182,
 0.6879795444438525,
 0.6593746256340495,
 0.7024418532543475,
 0.6553505035617467]

In [51]:
model9.evaluate_generator(train_gen9, steps = len(train_gen9))


[0.5833175691060355,
 0.6823416530552081,
 0.5933606064594181,
 0.7223555149363923,
 0.6207036236521531]

# Iteration 10

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:3.5}
epoch 20
150 x 150 images

Architecture from: https://www.pyimagesearch.com/2019/02/18/breast-cancer-classification-with-keras-and-deep-learning

In [31]:
batch_size = 10
image_dimensions = (150,150)
input_into_first_layer = (150,150,1)


train_gen10 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen10 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen10=train_gen10.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen10= test_gen10.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [32]:
train_gen10.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [33]:

# model9.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
# model9.add(Activation('relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Conv2D(32, (3, 3),activation = 'relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2) ))

# model9.add(Conv2D(64, (3, 3), activation ='relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model9.add(Dense(64, activation ='relu'))
# # model.add(Dropout(0.5))
# model9.add(Dense(1, activation ='sigmoid'))

# model9.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['binary_accuracy',recall,precision, f1])

# model9.summary()

# CONV => RELU => POOL
chanDim = -1

model10 = Sequential()

model10.add(SeparableConv2D(32, (3, 3), padding="same",
input_shape=input_into_first_layer))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(MaxPooling2D(pool_size=(2, 2)))
model10.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 2
model10.add(SeparableConv2D(64, (3, 3), padding="same"))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(SeparableConv2D(64, (3, 3), padding="same"))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(MaxPooling2D(pool_size=(2, 2)))
model10.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 3
model10.add(SeparableConv2D(128, (3, 3), padding="same"))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(SeparableConv2D(128, (3, 3), padding="same"))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(SeparableConv2D(128, (3, 3), padding="same"))
model10.add(Activation("relu"))
model10.add(BatchNormalization(axis=chanDim))
model10.add(MaxPooling2D(pool_size=(2, 2)))
model10.add(Dropout(0.25))

model10.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model10.add(Dense(256, activation ='relu'))
model10.add(BatchNormalization())

model10.add(Dropout(0.5))
model10.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))


model10.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model10.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_9 (Separabl (None, 150, 150, 32)      73        
_________________________________________________________________
activation_10 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
separable_conv2d_10 (Separab (None, 75, 75, 64)        2400      
_________________________________________________________________
activation_11 (Activation)   (None, 75, 75, 64)        0         
__________

In [34]:
model10.fit_generator(
        train_gen10,
        steps_per_epoch=3126,
        epochs=20,
        class_weight = {0:1,
                        1:3.5}
)

Epoch 1/20
3126/3126 [==============================] - 497s 159ms/step - loss: 1.1251 - binary_accuracy: 0.5719 - recall: 0.9391 - precision: 0.5467 - f1: 0.6710
Epoch 2/20
3126/3126 [==============================] - 487s 156ms/step - loss: 1.0566 - binary_accuracy: 0.5953 - recall: 0.9487 - precision: 0.5616 - f1: 0.6869
Epoch 3/20
3126/3126 [==============================] - 501s 160ms/step - loss: 0.9826 - binary_accuracy: 0.6423 - recall: 0.9493 - precision: 0.5931 - f1: 0.7108
Epoch 4/20
3126/3126 [==============================] - 484s 155ms/step - loss: 0.9710 - binary_accuracy: 0.6496 - recall: 0.9510 - precision: 0.5980 - f1: 0.7153
Epoch 5/20
3126/3126 [==============================] - 482s 154ms/step - loss: 0.9550 - binary_accuracy: 0.6574 - recall: 0.9547 - precision: 0.6040 - f1: 0.7206
Epoch 6/20
3126/3126 [==============================] - 486s 155ms/step - loss: 0.9601 - binary_accuracy: 0.6497 - recall: 0.9520 - precision: 0.5982 - f1: 0.7161
Epoch 7/20
3126/3126 [

In [43]:
model10.evaluate_generator(train_gen10, steps = len(train_gen10))


[0.5584337609514394,
 0.7444017909691262,
 0.9445523465587325,
 0.6687764754412804,
 0.7687074963244123]

In [44]:
model10.evaluate_generator(test_gen10, steps = len(test_gen10))


[0.5373099723359203,
 0.7595907934486409,
 0.9403087322211936,
 0.6650428349709572,
 0.7663180488149833]

# Iteration 11

class_mode=binary
loss=binary_crossentropy
outut layer: 1 sigmoid activation
class_weight = {0:1,
                        1:2.5}
epoch 20
150 x 150 images

Architecture from: https://www.pyimagesearch.com/2019/02/18/breast-cancer-classification-with-keras-and-deep-learning

In [20]:
batch_size = 10
image_dimensions = (150,150)
input_into_first_layer = (150,150,1)


train_gen11 = ImageDataGenerator(rescale = 1./255, rotation_range = 45, horizontal_flip = True, vertical_flip = True)
test_gen11 = ImageDataGenerator(rescale = 1./255)

# train_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset ='training')
# test_gen = im_gen.flow_from_directory(directory = '..\Images\All_images_in_classes', target_size=(50,50), color_mode='rgb', class_mode = 'categorical', batch_size =100, subset = 'validation')

train_gen11=train_gen11.flow_from_dataframe(dataframe=df_train, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)
test_gen11= test_gen11.flow_from_dataframe(dataframe=df_test, directory ='..\Images\All_Images', x_col='image_id', y_col='dx',target_size=image_dimensions, color_mode='grayscale', class_mode='binary', batch_size= batch_size)


Found 3126 images belonging to 2 classes.
Found 782 images belonging to 2 classes.


In [21]:
train_gen11.class_indices

{'0_not_cancer': 0, '1_cancer': 1}

In [22]:

# model9.add(Conv2D(32, (3, 3),input_shape=input_into_first_layer))
# model9.add(Activation('relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Conv2D(32, (3, 3),activation = 'relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2) ))

# model9.add(Conv2D(64, (3, 3), activation ='relu'))
# model9.add(MaxPooling2D(pool_size=(2, 2)))

# model9.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model9.add(Dense(64, activation ='relu'))
# # model.add(Dropout(0.5))
# model9.add(Dense(1, activation ='sigmoid'))

# model9.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['binary_accuracy',recall,precision, f1])

# model9.summary()

# CONV => RELU => POOL
chanDim = -1

model11 = Sequential()

model11.add(SeparableConv2D(32, (3, 3), padding="same",
input_shape=input_into_first_layer))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(MaxPooling2D(pool_size=(2, 2)))
model11.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 2
model11.add(SeparableConv2D(64, (3, 3), padding="same"))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(SeparableConv2D(64, (3, 3), padding="same"))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(MaxPooling2D(pool_size=(2, 2)))
model11.add(Dropout(0.25))
 
# (CONV => RELU => POOL) * 3
model11.add(SeparableConv2D(128, (3, 3), padding="same"))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(SeparableConv2D(128, (3, 3), padding="same"))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(SeparableConv2D(128, (3, 3), padding="same"))
model11.add(Activation("relu"))
model11.add(BatchNormalization(axis=chanDim))
model11.add(MaxPooling2D(pool_size=(2, 2)))
model11.add(Dropout(0.25))

model11.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model11.add(Dense(256, activation ='relu'))
model11.add(BatchNormalization())

model11.add(Dropout(0.5))
model11.add(Dense(1, activation ='sigmoid'))
# model.add(Activation('softmax'))


model11.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy',recall,precision, f1])

model11.summary()



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_1 (Separabl (None, 150, 150, 32)      73        
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
separable_conv2d_2 (S

In [23]:
model11.fit_generator(
        train_gen11,
        steps_per_epoch=3126,
        epochs=20,
        class_weight = {0:1,
                        1:2.5}
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
3126/3126 [==============================] - 531s 170ms/step - loss: 0.9310 - binary_accuracy: 0.6446 - recall: 0.8955 - precision: 0.6025 - f1: 0.6992
Epoch 2/20
3126/3126 [==============================] - 475s 152ms/step - loss: 0.8595 - binary_accuracy: 0.6752 - recall: 0.9127 - precision: 0.6248 - f1: 0.7217
Epoch 3/20
3126/3126 [==============================] - 554s 177ms/step - loss: 0.8708 - binary_accuracy: 0.6649 - recall: 0.9128 - precision: 0.6160 - f1: 0.7156
Epoch 4/20
3126/3126 [==============================] - 595s 190ms/step - loss: 0.8297 - binary_accuracy: 0.6901 - recall: 0.9235 - precision: 0.6355 - f1: 0.7332
Epoch 5/20
3126/3126 [==============================] - 647s 207ms/step - loss: 0.8630 - binary_accuracy: 0.6668 - recall: 0.9246 - precision: 0.6146 - f1: 0.7186
Epoch 6/20
3126/3126 [==============================] - 523s 167ms/step - loss: 0.8498 - binary_accuracy: 0.6724 - recall: 0.9256 - preci